In [1]:
import os

In [2]:
%pwd

'/config/workspace/Wine-Quality-ML-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/config/workspace/Wine-Quality-ML-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# entity
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [6]:
from Wine_Quality_ML_Project.constants import *
from Wine_Quality_ML_Project.utils.common import read_yaml,create_directories 

In [7]:
class ConfigurationManager:
    
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation 
        params = self.params.LogisticRegression
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
        )

        return model_evaluation_config

In [8]:
import pandas as pd
from sklearn.metrics import accuracy_score,precision_score,recall_score
from urllib.parse import urlparse
import joblib
import numpy as np
from Wine_Quality_ML_Project.utils.common import save_json

In [9]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):

        acc = accuracy_score(actual,pred)
        return acc

    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]

        predicted_qualities = model.predict(test_x)

        acc = self.eval_metrics(test_y,predicted_qualities)

        scores = {"accuracy":acc}
        save_json(path=Path(self.config.metric_file_name),data=scores)

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.save_results()
except Exception as e:
    raise e

[2024-01-19 12:58:32,213:INFO:common:yaml file: config/config.yaml loaded]
[2024-01-19 12:58:32,215:INFO:common:yaml file: params.yaml loaded]
[2024-01-19 12:58:32,219:INFO:common:yaml file: schema.yaml loaded]
[2024-01-19 12:58:32,221:INFO:common:created directory at: artifacts]
[2024-01-19 12:58:32,222:INFO:common:created directory at: artifacts/model_evaluation]
[2024-01-19 12:58:32,275:INFO:common:json file saved at: artifacts/model_evaluation/metrics.json]
